In [ ]:
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
from PIL import Image
import shutil
import pandas as pd

# FLIGHT 3

In [ ]:
anomalies_object_df = pd.read_csv(r'/Volumes/Samsung_T5/Haak_data/Flight3/Anomalies_object.csv')
anomalies_land_df = pd.read_csv(r'/Volumes/Samsung_T5/Haak_data/Flight3/Anomalies_land.csv')
anomalies_all_df = pd.read_csv(r'/Volumes/Samsung_T5/Haak_data/Flight3/Anomalies_all.csv')
anomalies_true_df = pd.read_csv(r'/Volumes/Samsung_T5/Haak_data/Flight3/True.csv')

In [ ]:
directory = '/Volumes/Samsung_T5/Haak_data/Flight3/20170926_simple-naming/' # 14-bit

true_imgs = []
ano_object_imgs = []
ano_land_imgs = []

files = len(os.listdir(directory))
loops = files

for i, file in enumerate(os.listdir(directory)[:loops]):
    print(str(i) + '/' + str(files), end='\r')

    file_png = file.split('.')[0][1:] + '.png'
    isAnomObject = anomalies_object_df['Anomalies_object'].str.contains(file_png).sum()
    isAnomLand   = anomalies_land_df['Anomalies_land'].str.contains(file_png).sum()
    isTrue       = anomalies_true_df['True'].str.contains(file_png).sum()
    
    if isTrue:
        img = np.array(Image.open(os.path.join(directory, file)))
        true_imgs = [*true_imgs, img]
    
    '''
    if isAnomObject:
        img = np.array(Image.open(os.path.join(directory, file)))
        ano_object_imgs = [*ano_object_imgs, img]
    '''   
    if isAnomLand:
        img = np.array(Image.open(os.path.join(directory, file)))
        ano_land_imgs = [*ano_land_imgs, img]
    
        
true_imgs       = np.array(true_imgs)
ano_object_imgs = np.array(ano_object_imgs)
ano_land_imgs   = np.array(ano_land_imgs)

In [ ]:
max2 = 0
min2 = 1000000

for i, file in enumerate(os.listdir(directory)[:loops]):
    print(str(i) + '/' + str(files), end='\r')

    file_png = file.split('.')[0][1:] + '.png'
    img = np.array(Image.open(os.path.join(directory, file)))
    
    temp_max = img.max()
    if temp_max > max2:
        max2 = temp_max
        
    temp_min = img.min()
    if temp_min < min2:
        min2 = temp_min

In [ ]:
min_all = 1333
max_all = 3751

min_true = 2082
max_true = 2646

In [ ]:
#hist, edges = np.histogram(true_imgs[0].flatten(), density=True, bins=(max2-min2), range=(min2,max2) )
hist, edges = np.histogram(true_imgs[0].flatten(), density=True, bins=(max_true-min_true)+1, range=(min_true,max_true+1) )
figure(figsize=(24, 6))
plt.bar(edges[:-1], hist)

In [ ]:
# define subplot grid
import random

fig, axs = plt.subplots(nrows=8, ncols=6, figsize=(32, 24))
#fig, axs = plt.subplots(nrows=2, ncols=6, figsize=(32, 12))
fig.subplots_adjust(hspace=1)
fig.suptitle("Histogram - Ocean Surface", fontsize=32, y=0.95)

n_imgs = len(anomalies_true_df)
error = 0
prev_rand_frame = 0
prev_hist = 0

# loop through tickers and axes
for i, ax in enumerate(axs.ravel()):
    rand_frame = random.randint(0, n_imgs)
    
    hist, edges = np.histogram(true_imgs[rand_frame].flatten(), 
                               density=True, 
                               bins=(max_true-min_true)+1, 
                               range=(min_true,max_true+1) )
    ax.plot(edges[:-1], hist)

    if i > 0:
        error = 100*( np.sum( (prev_hist-hist).clip(min=0) ) )
        ax.set_title('{} | error(i-1 -> i): {:.2f}%'
                     .format(anomalies_true_df['True'][rand_frame], error))
    else:
        ax.set_title('{}'
                     .format(anomalies_true_df['True'][rand_frame]))
    
    prev_hist = hist
    prev_rand_frame = rand_frame
plt.show()

In [ ]:
# define subplot grid
import random

fig, axs = plt.subplots(nrows=8, ncols=6, figsize=(32, 24))
#fig, axs = plt.subplots(nrows=2, ncols=6, figsize=(32, 12))
fig.subplots_adjust(hspace=1)
fig.suptitle("Histogram - Ocean Surface", fontsize=32, y=0.95)

n_imgs = len(anomalies_land_df)
error = 0
prev_rand_frame = 0
prev_hist = 0

# loop through tickers and axes
for i, ax in enumerate(axs.ravel()):
    rand_frame = random.randint(0, n_imgs)
    
    hist, edges = np.histogram(ano_land_imgs[rand_frame].flatten(), 
                               density=True, 
                               bins=(max_true-min_true)+1, 
                               range=(min_true,max_true+1) )
    ax.plot(edges[:-1], hist)

    if i > 0:
        error = 100*( np.sum( (prev_hist-hist).clip(min=0) ) )
        ax.set_title('{} | error(i-1 -> i): {:.2f}%'
                     .format(anomalies_true_df['True'][rand_frame], error))
    else:
        ax.set_title('{}'
                     .format(anomalies_true_df['True'][rand_frame]))
    
    prev_hist = hist
    prev_rand_frame = rand_frame
plt.show()

In [ ]:
plt.figure(figsize=(32, 18))
plt.title("Histogram - Ocean Surface", fontsize=12, y=0.95)

frames = len(true_imgs)
# loop through tickers and axes

for i in range(frames):
    if not (i % 5):
        print(str(i) + '/' + str(frames), end='\r')
        hist, edges = np.histogram(true_imgs[i].flatten(), 
                                       density=True, 
                                       bins=(max_true-min_true)+1, 
                                       range=(min_true,max_true+1) )
        plt.plot(edges[:-1], hist, label=anomalies_true_df['True'][i], color='black', linewidth=4, alpha=0.1)

#plt.legend(prop={'size': 10}, ncol=10)
plt.show()

In [ ]:
frames = len(true_imgs)

hist_true = []
edges_true = []

for i in range(frames):
    print(str(i) + '/' + str(frames), end='\r')
    hist, edges = np.histogram(true_imgs[i].flatten(), 
                                   density=True, 
                                   bins=(max_true-min_true)+1, 
                                   range=(min_true,max_true+1) )

    hist_true = [*hist_true, hist]
    edges_true = [*edges_true, edges]

hist_true = np.array(hist_true)
edges_true = np.array(edges_true)

In [ ]:
frames = len(true_imgs)

hist_true = []
edges_true = []

for i in range(frames):
    print(str(i) + '/' + str(frames), end='\r')
    hist, edges = np.histogram(true_imgs[i].flatten(), 
                                   density=True, 
                                   bins=(max_true-min_true)+1, 
                                   range=(min_true,max_true+1) )

    hist_true = [*hist_true, hist]
    edges_true = [*edges_true, edges]

hist_true = np.array(hist_true)
edges_true = np.array(edges_true)

# DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN
X = hist_true
clust_DBSCAN = DBSCAN(eps=0.01, min_samples=2).fit(X)
print(min(clust_DBSCAN.labels_))
print(max(clust_DBSCAN.labels_))

labels_uint = clust_DBSCAN.labels_

In [ ]:
fig = plt.figure(figsize=(32, 18))
ax = fig.add_subplot(1, 1, 1)
plt.title("Histogram - Ocean Surface", fontsize=24, y=0.95)

# Colors
colors = plt.get_cmap('jet', max(labels_uint) + 1)
frames = len(true_imgs)
# loop through tickers and axes

for i in range(frames):
    if not (i % 10):
        print(str(i) + '/' + str(frames), end='\r')
        color_idx = colors(labels_uint[i])
        plt.plot(edges_true[0][:-1], hist_true[i], 
                 color=color_idx, linewidth=4, 
                 alpha=0.5)

for i in range(max(labels_uint) + 1):
    c = colors(i)
    plt.plot(edges_true[0][:-1], np.zeros(hist_true[0].shape), 
                 color=c, linewidth=10, 
                 alpha=0.5, label='class: ' + str(i))

plt.plot(edges[:-1], np.zeros(hist.shape), 
             color='white', linewidth=10, 
             alpha=1)
    
plt.legend(prop={'size': 24})
plt.show()

# OPTICS

In [ ]:
from sklearn.cluster import OPTICS
X = hist_true
clust_OPTICS = OPTICS(min_samples=50, xi=0.05, min_cluster_size=0.05).fit(X)
print(min(clust_OPTICS.labels_))
print(max(clust_OPTICS.labels_))

labels_uint = clust_OPTICS.labels_ + 1

In [ ]:
fig = plt.figure(figsize=(32, 18))
ax = fig.add_subplot(1, 1, 1)
plt.title("Histogram - Ocean Surface", fontsize=24, y=0.95)

# Colors
colors = plt.get_cmap('jet', max(labels_uint) + 1)
frames = len(true_imgs)
# loop through tickers and axes

for i in range(frames):
    if not (i % 10):
        print(str(i) + '/' + str(frames), end='\r')
        color_idx = colors(labels_uint[i])
        plt.plot(edges_true[0][:-1], hist_true[i], 
                 color=color_idx, linewidth=4, 
                 alpha=0.5)

for i in range(max(labels_uint) + 1):
    c = colors(i)
    plt.plot(edges_true[0][:-1], np.zeros(hist_true[0].shape), 
                 color=c, linewidth=10, 
                 alpha=0.5, label='class: ' + str(i))

plt.plot(edges[:-1], np.zeros(hist.shape), 
             color='white', linewidth=10, 
             alpha=1)
    
plt.legend(prop={'size': 24})
plt.show()

# K-MEANS

In [ ]:
from sklearn.cluster import KMeans
X = hist_true
clust_KMeans = KMeans(n_clusters=3).fit(X)
print(min(clust_KMeans.labels_))
print(max(clust_KMeans.labels_))

labels_uint = clust_KMeans.labels_

In [ ]:
fig = plt.figure(figsize=(32, 18))
ax = fig.add_subplot(1, 1, 1)
plt.title("Histogram - Ocean Surface", fontsize=24, y=0.95)


# Colors
colors = plt.get_cmap('jet', max(labels_uint) + 1)
frames = len(true_imgs)
# loop through tickers and axes

for i in range(frames):
    if not (i % 10):
        print(str(i) + '/' + str(frames), end='\r')
        color_idx = colors(labels_uint[i])
        plt.plot(edges_true[0][:-1], hist_true[i], 
                 color=color_idx, linewidth=4, 
                 alpha=0.5)

for i in range(max(labels_uint) + 1):
    c = colors(i)
    plt.plot(edges_true[0][:-1], np.zeros(hist_true[0].shape), 
                 color=c, linewidth=10, 
                 alpha=0.5, label='class: ' + str(i))

plt.plot(edges[:-1], np.zeros(hist.shape), 
             color='white', linewidth=10, 
             alpha=1)
    
plt.legend(prop={'size': 24})
plt.show()

# True & Land

In [ ]:
X_true_n_land = np.append(true_imgs, ano_land_imgs, axis=0)

X_true_label = np.zeros(true_imgs.shape[0])
X_ano_land_label = np.ones(ano_land_imgs.shape[0])
X_true_n_land_labels = np.append(X_true_label, X_ano_land_label)

X_true_n_land.shape

In [ ]:
frames = len(X_true_n_land)

hist_true_n_land = []
edges_true_n_land = []

for i in range(frames):
    print(str(i) + '/' + str(frames), end='\r')
    hist, edges = np.histogram(X_true_n_land[i].flatten(), 
                                   density=True, 
                                   bins=(max_true-min_true)+1, 
                                   range=(min_true,max_true+1) )

    hist_true_n_land = [*hist_true_n_land, hist]
    edges_true_n_land = [*edges_true_n_land, edges]

hist_true_n_land = np.array(hist_true_n_land)
edges_true_n_land = np.array(edges_true_n_land)

hist_true_n_land.shape

In [ ]:
X_true_n_land_flat = np.reshape(X_true_n_land, (X_true_n_land.shape[0], -1))

# t-SNE - true & land

In [ ]:
from sklearn.manifold import TSNE
X = hist_true_n_land
X_TSNE = TSNE(n_components=2).fit_transform(X)

print(X_TSNE[:,0].shape)
print(X_TSNE[:,1].shape)

In [ ]:
colors = plt.get_cmap('jet', 2)

plt.figure(figsize=(4, 3))
plt.scatter(X_TSNE[:,0], X_TSNE[:,1], c=colors(X_true_n_land_labels), s=0.1)
plt.show()

In [ ]:
from sklearn.manifold import TSNE
X = X_true_n_land_flat
X_TSNE = TSNE(n_components=2).fit_transform(X)

print(X_TSNE[:,0].shape)
print(X_TSNE[:,1].shape)

In [ ]:
colors = plt.get_cmap('rainbow', 2)

plt.figure(figsize=(4, 3))
plt.scatter(X_TSNE[:,0], X_TSNE[:,1], c=colors(X_true_n_land_labels), s=0.1)
plt.show()

# t-SNE

In [ ]:
%matplotlib notebook
%matplotlib widget
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(5, 4))
ax = Axes3D(fig)
ax.scatter(X_TSNE[:,0], X_TSNE[:,1], X_TSNE[:,2], s=5, c=colors(X_true_n_land_labels))

plt.show()

In [ ]:
from sklearn.manifold import TSNE
X = hist_true
X_TSNE = TSNE(n_components=3).fit_transform(X)

In [ ]:
%matplotlib notebook
%matplotlib widget
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(5, 4))
ax = Axes3D(fig)
ax.scatter(X_TSNE[:,0], X_TSNE[:,1], X_TSNE[:,2], s=5)

plt.show()

# t-SNE + DBSCAN

In [ ]:
from sklearn.cluster import KMeans
X = X_TSNE
clust_tSNE_KMeans = KMeans().fit(X)
print(min(clust_tSNE_KMeans.labels_))
print(max(clust_tSNE_KMeans.labels_))

labels_uint = clust_tSNE_KMeans.labels_

%matplotlib notebook
%matplotlib widget
from mpl_toolkits.mplot3d import Axes3D

colors = plt.get_cmap('jet', max(labels_uint) + 1)

fig = plt.figure(figsize=(5, 4))
ax = Axes3D(fig)

ax.scatter(X_TSNE[:,0], X_TSNE[:,1], X_TSNE[:,2], c=colors(labels_uint), s=5)
plt.show()

# PCA

In [ ]:
from sklearn.decomposition import PCA
%matplotlib inline

X = hist_true
X_PCA = PCA().fit(X)

X_embedded = X_PCA.transform(X)
X_reconstructed = X_PCA.inverse_transform(X_embedded)

components = 20
plt.figure(figsize=(16, 6))
plt.bar(np.arange(1,components+1),X_PCA.explained_variance_ratio_[:components])
plt.plot(np.arange(1,components+1), np.cumsum(X_PCA.explained_variance_ratio_[:components]),
         color="orange", marker="D")
plt.xticks(np.arange(1,components+1))
plt.show()

In [ ]:
from sklearn.decomposition import PCA

components = 3
X = hist_true
X_PCA = PCA(n_components=components).fit(X)

X_embedded = X_PCA.transform(X)
X_reconstructed = X_PCA.inverse_transform(X_embedded)

In [ ]:
plt.figure(figsize=(16, 6))
plt.bar(np.arange(1,components+1),X_PCA.explained_variance_ratio_[:components])
plt.plot(np.arange(1,components+1), np.cumsum(X_PCA.explained_variance_ratio_[:components]),
         color="orange", marker="D")
plt.xticks(np.arange(1,components+1))
plt.show()

# PCA + K-MEANS

In [ ]:
from sklearn.cluster import KMeans

X = X_reconstructed
clust_KMeans = KMeans(n_clusters=components).fit(X)
print(min(clust_KMeans.labels_))
print(max(clust_KMeans.labels_))

labels_uint = clust_KMeans.labels_

In [ ]:
fig = plt.figure(figsize=(16, 12))
ax = fig.add_subplot(1, 1, 1)
plt.title("Histogram - Ocean Surface - Reconstructed", fontsize=24, y=0.95)


# Colors
colors = plt.get_cmap('jet', max(labels_uint) + 1)
frames = len(true_imgs)
# loop through tickers and axes

for i in range(frames+1):
    if not (i % 10):
        print(str(i) + '/' + str(frames), end='\r')
        color_idx = colors(labels_uint[i])
        plt.plot(edges_true[0][:-1], X_reconstructed[i], 
                 color=color_idx, linewidth=4, 
                 alpha=0.5)

for i in range(max(labels_uint) + 1):
    c = colors(i)
    plt.plot(edges_true[0][:-1], np.zeros(hist_true[0].shape), 
                 color=c, linewidth=10, 
                 alpha=0.5, label='class: ' + str(i))

plt.plot(edges[:-1], np.zeros(hist.shape), 
             color='white', linewidth=10, 
             alpha=1)
    
plt.legend(prop={'size': 24})
plt.show()

In [ ]:
fig = plt.figure(figsize=(16, 12))
ax = fig.add_subplot(1, 1, 1)
plt.title("Histogram - Ocean Surface - True", fontsize=24, y=0.95)


# Colors
colors = plt.get_cmap('jet', max(labels_uint) + 1)
frames = len(true_imgs)
# loop through tickers and axes

for i in range(frames+1):
    if not (i % 10):
        print(str(i) + '/' + str(frames), end='\r')
        color_idx = colors(labels_uint[i])
        plt.plot(edges_true[0][:-1], hist_true[i], 
                 color=color_idx, linewidth=4, 
                 alpha=0.5)

for i in range(max(labels_uint) + 1):
    c = colors(i)
    plt.plot(edges_true[0][:-1], np.zeros(hist_true[0].shape), 
                 color=c, linewidth=10, 
                 alpha=0.5, label='class: ' + str(i))

plt.plot(edges[:-1], np.zeros(hist.shape), 
             color='white', linewidth=10, 
             alpha=1)
    
plt.legend(prop={'size': 24})
plt.show()